In [1]:
import os
import os.path
import sys
from pathlib import Path
import pandas as pd
sys.path.insert(0, os.path.join(Path.home(), 'Projects', 'elite-tools', 'src'))
import eddb
eddb.load_feeds()

Using existing file "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-systems_populated.jsonl" for Feeds.POPULATED_SYSTEMS.
# Feeds.POPULATED_SYSTEMS records loaded:  20629
Using existing file "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-factions.jsonl" for Feeds.FACTIONS.
# Feeds.FACTIONS records loaded:  77091


In [2]:
origin = 'Azrael'
faction = "The Order of Mobius"
core_systems = set(eddb.query_systems_by_faction(faction))
extended_systems = set([inf_s for core_s in core_systems for inf_s in eddb.query_nearby_systems(core_s, eddb.SYSTEM_INFLUENCE_RANGE)])

In [3]:
cpf = eddb.controlling_player_factions(extended_systems)
"\n".join(cpf)

"The Order of Mobius\nImperial Trade Works\nStellar Winter Squadron\nOuter Worlds Network\nDaily Routine Incorporated\nBlack Ram\nLibertas Co-operative\nLavigny's Legion\nSilver Hunter Order\nZeta Corporation\nEmpire Corsairs\nImperial Ghost Division\nImperial Privateers\nThe Independent Light Wheel\nThe 10th Legion\nRoyal Elite of the Grand Duke\nSociety Of Greybeards\nExtra Corporation\nMagnates of Cemiess\nBrowncoat Uprising\nDecimus Imperium Lex\nPrismatic Knights\nThe Empires Fleet\nImperial Galactic Investments Inc\nThe Guardian Angels\nAncient World Order\nThe White Templars\nWhite Scar Contingency"

In [4]:
faction_stats = {}
for s_name in extended_systems:
    s = eddb.populated_systems[s_name]
    cf_name = s['controlling_minor_faction']
    if cf_name in eddb.player_faction_names:
        if cf_name not in faction_stats.keys():
            faction_stats[cf_name] = {'name': cf_name, 'control': 0, 'presence': 0}
        faction_stats[cf_name]['control'] += 1
    for f in s['minor_faction_presences']:
        f_name = eddb.faction_names_by_id[f['minor_faction_id']]
        if f_name in eddb.player_faction_names:
            if f_name not in faction_stats.keys():
                faction_stats[f_name] = {'name': f_name, 'control': 0, 'presence': 0}
            if f_name != cf_name:
                faction_stats[f_name]['presence'] += 1

pd.DataFrame.from_dict(faction_stats.values()).set_index('name').sort_values(by=['control', 'presence', 'name'], ascending=[False, False, True])

,control,presence
name,,
The Order of Mobius,21,1
Lavigny's Legion,14,1
Decimus Imperium Lex,13,0
Silver Hunter Order,8,3
The 10th Legion,8,0
Daily Routine Incorporated,7,0
Libertas Co-operative,6,3
The White Templars,5,0
Empire Corsairs,4,4


In [5]:
faction_report = eddb.summarize_faction_systems(extended_systems, origin).sort_values(by=['Priority', 'Name'], ascending=[True, True])
faction_report

,Category,Name,Distance,Controlling Faction,Happiness,Influence,Player Factions Present,States,Population,Government,Allegiance,Security Level,Primary Economy
Priority,,,,,,,,,,,,,
0,Insurgency,Dvorsi,14.5,The Order of Mobius,2,39.6000,"The Order of Mobius, The Guardians",Boom,65081959,Cooperative,Independent,High,High Tech
0,Insurgency,Njiri,12.0,The Order of Mobius,2,51.0490,"The Order of Mobius, Black Ram",Boom,17186307,Cooperative,Independent,Medium,High Tech
1,Control,Apathaam,6.0,The Order of Mobius,2,56.3000,The Order of Mobius,Boom,3506390,Cooperative,Independent,Medium,Industrial
1,Control,Asase Ya,24.6,The Order of Mobius,2,45.7000,The Order of Mobius,None,10832961797,Cooperative,Independent,High,Agriculture
1,Control,Azrael,0.0,The Order of Mobius,1,52.5475,The Order of Mobius,"Boom, Civil Liberty",4500000,Cooperative,Independent,Medium,Extraction
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,NPC,Tewi,23.2,Tewi Guardians of Tradition,2,30.4000,,None,310841,Patronage,Empire,Medium,Refinery
5,NPC,Thanatos,31.0,Thanatos Inc,2,53.0531,,Boom,450000,Corporate,Empire,Medium,Refinery
5,NPC,Wandjin,23.8,HR 571 Emperor's Grace,2,44.5445,,Boom,114523,Patronage,Empire,Medium,Extraction


In [6]:
faction_report.to_clipboard()